In [86]:
import numpy as np
import matplotlib.pyplot as plt
from openpiv import tools, pyprocess, validation, filters, scaling 
from skimage.io import imread
from skimage.filters import difference_of_gaussians
from glob import glob
from tqdm import tqdm

In [17]:
%matplotlib qt

In [77]:
movie_dir = r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_05_30-dpt-gfp_r4-gal4_ecoli-hs-dtom_4hrs_flow_field/larva_1/movie_9_dual_tifs_cropped_culled'
name_str = '*.tif'

In [78]:
frame_a = imread(movie_dir + '/im_056.tif')
frame_b = imread(movie_dir + '/im_057.tif')

#frame_a = (2 ** 16 - 1) * difference_of_gaussians(frame_a, low_sigma=1)
#frame_b = (2 ** 16 - 1) * difference_of_gaussians(frame_b, low_sigma=1)

# plt.figure()
# plt.imshow(frame_a)

In [108]:
winsize = 16 # pixels, interrogation window size in frame A
searchsize = 20  # pixels, search in image B
overlap = 4 # pixels, 50% overlap
dt = 0.03 # sec, time interval between pulses


u0, v0, sig2noise = pyprocess.extended_search_area_piv(frame_a.astype(np.int32), 
                                                       frame_b.astype(np.int32), 
                                                       window_size=winsize, 
                                                       overlap=overlap, 
                                                       dt=dt, 
                                                       search_area_size=searchsize, 
                                                       sig2noise_method='peak2peak')



flags = validation.sig2noise_val( sig2noise, 
                                 threshold = 1.15 )

thresh = 80
#flags = validation.global_val(u0, v0, u_thresholds=[-thresh, thresh], v_thresholds=[-thresh, thresh])


x, y = pyprocess.get_coordinates( image_size=frame_a.shape, 
                                 search_area_size=searchsize, 
                                 overlap=overlap )

u2, v2 = filters.replace_outliers( u0, v0, 
                                   flags,
                                   method='localmean', 
                                   max_iter=3, 
                                   kernel_size=3)

In [109]:
plt.figure()
plt.quiver(x, y, u2, v2)

In [85]:
def get_vector_field(frame_a, frame_b):
    winsize = 32 # pixels, interrogation window size in frame A
    searchsize = 38  # pixels, search in image B
    overlap = 16 # pixels, 50% overlap
    dt = 0.03 # sec, time interval between pulses


    u0, v0, sig2noise = pyprocess.extended_search_area_piv(frame_a.astype(np.int32), 
                                                           frame_b.astype(np.int32), 
                                                           window_size=winsize, 
                                                           overlap=overlap, 
                                                           dt=dt, 
                                                           search_area_size=searchsize, 
                                                           sig2noise_method='peak2peak')



    flags = validation.sig2noise_val( sig2noise, 
                                     threshold = 1.05 )


   

    u2, v2 = filters.replace_outliers( u0, v0, 
                                       flags,
                                       method='localmean', 
                                       max_iter=3, 
                                       kernel_size=3)
    
    return u2, v2

In [87]:
files = sorted(glob(movie_dir + '/' + name_str))
for i in tqdm(range(len(files) - 1)):
    frame_a = imread(files[i])
    frame_b = imread(files[i+1])
    u, v = get_vector_field(frame_a, frame_b)
    if i == 0:
        u_arr = np.zeros(((len(files) - 1), u.shape[0], u.shape[1]))
        v_arr = np.zeros(((len(files) - 1), u.shape[0], u.shape[1]))
    u_arr[i] = u
    v_arr[i] = v
    
x, y = pyprocess.get_coordinates( image_size=frame_a.shape, 
                                 search_area_size=38, 
                                 overlap=16 )

100%|█████████████████████████████████████████| 200/200 [01:38<00:00,  2.03it/s]


In [98]:
mean_u = np.mean(u_arr, axis=0)
mean_v = np.mean(v_arr, axis=0)
std_u = np.std(u_arr, axis=0)
std_v = np.std(v_arr, axis=0)

#thresh = 0.1
#mean_u[std_u / mean_u > thresh] = np.nan
#mean_v[std_v / mean_v > thresh] = np.nan

plt.figure()
plt.quiver(x, y, mean_u, mean_v)

plt.figure()
plt.pcolor(x, y, np.log10(np.sqrt(std_u ** 2 / mean_u ** 2 + std_v ** 2 / mean_v ** 2)))
plt.colorbar()

In [99]:
plt.figure()
plt.quiver(x, y, u_arr[0], v_arr[0])

plt.figure()
plt.quiver(x, y, u_arr[1], v_arr[1])

In [101]:
median_u = np.median(u_arr[:80], axis=0)
median_v = np.median(v_arr[:80], axis=0)

plt.figure()
plt.quiver(x, y, median_u, median_v)